In [2]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Dashboard components
import dash_leaflet as dl
from dash import dcc, html
from dash import dash_table
from dash.dependencies import Input, Output
import plotly.express as px
import base64
JupyterDash.infer_jupyter_proxy_config()

# Data / utils
import pandas as pd

# CRUD module (Project One)
from CRUD_Python_Module import AnimalShelter

# ------------------------------
# Data (Model) — connect & load
# ------------------------------
username = "aacuser"
password = "Rose01"

# If your AnimalShelter takes username/password positionally, this is fine.
db = AnimalShelter(username, password)

# Load all docs
df = pd.DataFrame.from_records(db.read({}))

# Drop Mongo _id (ObjectId not JSON-serializable for DataTable)
if "_id" in df.columns:
    df.drop(columns=["_id"], inplace=True)
   


# ------------------------------
# App (View)
# ------------------------------
app = JupyterDash(__name__)

# Logo
image_filename = "Grazioso Salvare Logo.png"
with open(image_filename, "rb") as f:
    encoded_logo = base64.b64encode(f.read()).decode()

# Filter radio buttons
filter_component = dcc.RadioItems(
    id="filter-type",
    options=[
        {"label": "Water Rescue", "value": "water"},
        {"label": "Mountain/Wilderness Rescue", "value": "mountain"},
        {"label": "Disaster/Individual Tracking", "value": "disaster"},
        {"label": "Reset (show all)", "value": "reset"},
    ],
    value="reset",
    labelStyle={"display": "inline-block", "marginRight": "16px"},
    inputStyle={"marginRight": "6px"},
)

app.layout = html.Div([
    # Header with logo + unique identifier
    html.Div(style={"display": "flex", "alignItems": "center", "gap": "16px"}, children=[
        html.Img(src=f"data:image/png;base64,{encoded_logo}", style={"height": "48px"}),
        html.H1("CS-340 Dashboard — Caleb McManus", style={"margin": 0}),
    ]),
    html.Hr(),

    html.Div([html.H4("Filter options"), filter_component]),
    html.Hr(),

    dash_table.DataTable(
        id="datatable-id",
        columns=[{"name": c, "id": c, "deletable": False, "selectable": True} for c in df.columns],
        data=df.to_dict("records"),

        # interactivity
        page_action="native",
        page_current=0,
        page_size=10,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        row_selectable="single",
        selected_rows=[0],

        # --- make sure the table body is visible ---
        style_table={
            "overflowX": "auto",
            "overflowY": "auto",
            "height": "520px",      # <- fixed height for the table body
            "maxHeight": "520px",
        },
        style_cell={
            "minWidth": "120px",
            "width": "120px",
            "maxWidth": "260px",
            "whiteSpace": "normal",
            "textAlign": "left",
        },
        style_header={"fontWeight": "bold"},
    ),

    html.Br(),
    html.Hr(),

    html.Div(style={"display": "flex", "gap": "24px", "flexWrap": "wrap"}, children=[
        html.Div(id="graph-id", style={"flex": "1 1 420px"}),
        html.Div(id="map-id",   style={"flex": "1 1 420px"}),
    ]),
])

# ------------------------------
# Controller (Callbacks)
# ------------------------------
def _or_regex(field, terms):
    """Return a Mongo $or clause of case-insensitive 'contains' regexes."""
    return {"$or": [{field: {"$regex": term, "$options": "i"}} for term in terms]}

def query_for_filter(filter_type: str):
    dog = {"animal_type": "Dog"}

    if filter_type == "water":
        breeds = ["Labrador Retriever", "Chesapeake Bay Retriever", "Newfoundland"]
        return {
            "$and": [
                dog,
                _or_regex("breed", breeds),
                {"sex_upon_outcome": {"$regex": r"^Intact Female$", "$options": "i"}},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}},
            ]
        }

    if filter_type == "mountain":
        breeds = [
            "German Shepherd",
            "Alaskan Malamute",
            "Old English Sheepdog",
            "Siberian Husky",
            "Rottweiler",
        ]
        return {
            "$and": [
                dog,
                _or_regex("breed", breeds),
                {"sex_upon_outcome": {"$regex": r"^Intact Male$", "$options": "i"}},
                {"age_upon_outcome_in_weeks": {"$gte": 26, "$lte": 156}},
            ]
        }

    if filter_type == "disaster":
        breeds = ["Doberman Pinscher", "German Shepherd", "Golden Retriever", "Bloodhound", "Rottweiler"]
        return {
            "$and": [
                dog,
                _or_regex("breed", breeds),
                {"sex_upon_outcome": {"$regex": r"^Intact Male$", "$options": "i"}},
                {"age_upon_outcome_in_weeks": {"$gte": 20, "$lte": 300}},
            ]
        }

    # reset (unfiltered) -> let read() return all docs
    return None

@app.callback(
    Output("datatable-id", "data"),
    [Input("filter-type", "value")]
)
def update_dashboard(filter_type):
    q = query_for_filter(filter_type)

    # Fresh connection inside the callback (avoids stale client in some Codio setups)
    conn = AnimalShelter(username, password)

    # If your read() expects None for "all", pass None; otherwise pass {}
    docs = conn.read(q if q is not None else None)

    dff = pd.DataFrame.from_records(docs)
    if not dff.empty and "_id" in dff.columns:
        dff.drop(columns=["_id"], inplace=True)

    print("filter:", filter_type, "rows:", 0 if dff.empty else len(dff))
    return dff.to_dict("records")

# Chart reflects the current table view
@app.callback(
    Output("graph-id", "children"),
    [Input("datatable-id", "derived_virtual_data")]
)
def update_graphs(viewData):
    dff = pd.DataFrame.from_dict(viewData) if viewData else df.copy()

    if dff.empty or ("breed" not in dff.columns):
        fig = px.bar(title="No data to display")
    else:
        # Top N + "Other" to avoid tiny unreadable slices
        counts = dff["breed"].fillna("Unknown").value_counts()
        top_n = 8
        top = counts.head(top_n)
        other = counts.iloc[top_n:].sum()
        if other and other > 0:
            top = pd.concat([top, pd.Series({"Other": other})])

        # Solid pie (no hole), percent labels inside, legend on the side
        fig = px.pie(
            values=top.values,
            names=top.index,
            title="Breed Distribution — current table"
        )
        fig.update_traces(
            textposition="inside",
            textinfo="percent",           # show % on slices; names live in the legend
            hovertemplate="%{label}: %{value} animals<br>%{percent}"
        )
        fig.update_layout(
            showlegend=True,
            legend_title_text="Breed",
            margin=dict(l=0, r=0, t=40, b=0),
            height=420,
            uniformtext_minsize=10,       # hide tiny labels instead of overlapping
            uniformtext_mode="hide"
        )

    return dcc.Graph(figure=fig)

# Highlight selected columns (guard for None on first render)
@app.callback(Output("datatable-id", "style_data_conditional"),
              [Input("datatable-id", "selected_columns")])
def update_styles(selected_columns):
    if not selected_columns:
        return []
    return [{'if': {'column_id': col}, 'background_color': '#D2F3FF'} for col in selected_columns]

# Map reflects the selected row
@app.callback(Output("map-id", "children"),
              [Input("datatable-id", "derived_virtual_data"),
               Input("datatable-id", "derived_virtual_selected_rows")])
def update_map(viewData, index):
    dff = pd.DataFrame.from_dict(viewData) if viewData else df.copy()

    # Empty → base map
    if dff.empty:
        return [dl.Map(style={"width": "100%", "height": "500px"},
                       center=[30.75, -97.48], zoom=10,
                       children=[dl.TileLayer(id="base-layer-id")])]

    # Default to first row if none selected
    row = (index or [0])[0]
    row = max(0, min(row, len(dff) - 1))

    # Prefer named columns; else fallback to iloc positions used in the course
    def safe_get(col_name, iloc_idx, default=None):
        if col_name in dff.columns:
            return dff.iloc[row][col_name]
        try:
            return dff.iloc[row, iloc_idx]
        except Exception:
            return default

    lat   = safe_get("location_lat",  13, 30.75)
    lon   = safe_get("location_long", 14, -97.48)
    breed = safe_get("breed",          4, "Unknown")
    name  = safe_get("name",           9, "Unknown")

    return [dl.Map(
        style={"width": "100%", "height": "500px"},
        center=[30.75, -97.48], zoom=10,
        children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[float(lat), float(lon)], children=[
                dl.Tooltip(str(breed)),
                dl.Popup([html.H1("Animal Name"), html.P(str(name))])
            ])
        ]
    )]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on https://megarider-pagecitizen-3000.codio.io/proxy/8050/
filter: reset rows: 10008
filter: water rows: 25
filter: reset rows: 10008
filter: mountain rows: 19
filter: disaster rows: 28
filter: reset rows: 10008
filter: water rows: 25
filter: mountain rows: 19
filter: disaster rows: 28
